In [0]:
from pyspark.sql import functions as F, types as T
from delta.tables import DeltaTable
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *

#Functions

In [0]:
def add_fixture_key(df, 
                    season_col="season", 
                    fixture_col="fixture"):
    return df.withColumn("fixture_key",
        F.concat(
            F.lit("20"), 
            F.regexp_replace(F.col(season_col), "_", ""), 
            F.lpad(F.col(fixture_col).cast("string"), 3, "0")
        )
    )

def load_fixtures(silver_schema: str):
    return spark.table(f"{silver_schema}.fixtures"
        ).select(
            "fixture_key", 
            "home_team_key", 
            "away_team_key"
        )

In [0]:
def get_players(bronze_schema: str, 
                silver_schema: str, 
                seasons: list, 
                source_type: str) -> DataFrame:
    """
    Unified loader for player data from either 'HIST' (historic) or 'API' source.
    Returns a DataFrame with player_id, player_key, season_key, player_season_key,
    first_name, second_name, team_key, initial_value, current_value,
    first_fixture_key, last_fixture_key.
    """
    
    fixtures_df = load_fixtures(silver_schema)
    player_records = []

    for season in seasons:
        season_key = "20" + season.replace("_", "")

        # Load player metadata
        if source_type == "HIST":
            player_meta = spark.table(f"{bronze_schema}.players_raw_{season}").select(
                F.col("id").alias("player_meta_id"),
                "first_name",
                "second_name",
                F.col("code").alias("player_key"),
                "element_type"
            )
        elif source_type == "API":
            player_meta = spark.table(f"{bronze_schema}.elements_{season}").select(
                F.col("id").alias("player_meta_id"),
                "first_name",
                "second_name",
                F.col("code").alias("player_key"),
                "element_type"
            )
        else:
            raise ValueError(f"Unsupported source_type: {source_type}")

        # Load player stats
        stats_df = spark.table(f"{bronze_schema}.player_gameweek_stats_{season}").select(
            F.col("element").alias("player_id"),
            "was_home",
            "fixture",
            "round",
            "value"
        ).withColumn("season", F.lit(season))

        stats_df = add_fixture_key(stats_df)

        # Join player_key
        stats_df = stats_df.join(
            player_meta.select("player_meta_id", "player_key"),
            on = stats_df["player_id"] == player_meta["player_meta_id"],
           how =  "left"
        )

        # Join with fixtures
        stats_df = stats_df.join(fixtures_df, 
                                 on = "fixture_key", 
                                 how = "left").withColumn(
            "team_key",
            F.when(F.col("was_home"), F.col("home_team_key")).otherwise(F.col("away_team_key"))
        )

        # Add season keys
        stats_df = stats_df.withColumn("season_key", F.lit(season_key)).withColumn(
            "player_season_key",
            F.concat(F.lit(season_key), F.lpad(F.col("player_id").cast("string"), 3, "0"))
        )

        # Aggregate
        grouped_df = stats_df.groupBy("player_key", "team_key", "season_key", "player_season_key").agg(
            F.first("player_id").alias("player_id"),
            F.first("value").alias("initial_value"),
            F.last("value").alias("current_value"),
            F.min("fixture_key").alias("first_fixture_key"),
            F.max("fixture_key").alias("last_fixture_key")
        )

        # Join back metadata
        final_df = grouped_df.join(
            player_meta.select("player_key", "first_name", "second_name"),
            on = "player_key",
            how = "left"
        ).select(
            "player_id", "player_key", "season_key", "player_season_key",
            "first_name", "second_name", "team_key",
            "initial_value", "current_value",
            "first_fixture_key", "last_fixture_key"
        )

        player_records.append(final_df)

    # Union all seasons
    silver_players_df = player_records[0]
    for df in player_records[1:]:
        silver_players_df = silver_players_df.unionByName(df)

    return silver_players_df

In [0]:
def write_to_table(
    df: DataFrame,
    table_name: str,
    mode: str = "overwrite",
    merge_schema: bool = True,
    partition_by: list[str] = None,
    path: str = None,
    save_as_table: bool = True
) -> None:
    """
    Generalised Delta write helper for bronze layer.

    Parameters:
    - df (DataFrame): Spark DataFrame to write.
    - table_name (str): Name of the Delta table (used if save_as_table=True).
    - mode (str): Write mode ('overwrite', 'append', 'ignore', 'error', etc.).
    - merge_schema (bool): Whether to merge schema on write.
    - partition_by (list[str], optional): List of columns to partition by.
    - path (str, optional): Path to save the Delta table (used if save_as_table=False).
    - save_as_table (bool): If True, saves as managed table; else saves to path.

    Raises:
    - ValueError: If neither save_as_table nor path is properly specified.
    """

    df_with_ts = df.withColumn("last_updated", F.current_timestamp())

    writer = df_with_ts.write.format("delta").mode(mode)

    if merge_schema:
        writer = writer.option("mergeSchema", "true")
    elif mode == "overwrite":
        writer = writer.option("overwriteSchema", "true")

    if partition_by:
        writer = writer.partitionBy(*partition_by)

    if save_as_table:
        writer.saveAsTable(table_name)
    elif path:
        writer.save(path)
    else:
        raise ValueError("Either save_as_table must be True or a path must be provided.")

In [0]:
def merge_to_table(
    df: DataFrame,
    table_name: str,
    merge_condition: str,
    spark: SparkSession,
    partition_by: list[str] = None
) -> None:
    """
    Performs an upsert (merge) into a Delta table.

    Parameters:
    - df (DataFrame): Incoming DataFrame to merge.
    - table_name (str): Target Delta table name.
    - merge_condition (str): SQL condition for matching rows.
    - spark (SparkSession): Active Spark session.
    - partition_by (list[str], optional): Columns to partition by on initial write.

    If the table does not exist, it will be created using write_to_table.
    """
    df_with_ts = df.withColumn("last_updated", F.current_timestamp())

    if not spark.catalog.tableExists(table_name):
        write_to_table(
            df=df_with_ts,
            table_name=table_name,
            partition_by=partition_by
        )
    else:
        delta_table = DeltaTable.forName(spark, table_name)
        (
            delta_table.alias("target")
            .merge(
                source=df_with_ts.alias("source"),
                condition=merge_condition
            )
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )

#Variables

In [0]:
try:
    ENV = dbutils.widgets.get("ENV")
except Exception:
    ENV = "dev"

try:
    PROTOCOL = dbutils.widgets.get("PROTOCOL")
except Exception:
    PROTOCOL = "HIST"

#ensure valid ENV and PROTOCOL
valid_envs = {"dev", "test", "prod"}
valid_protocols = {"HIST", "INCR"}

# Validate ENV
if ENV not in valid_envs:
    print(f"Invalid ENV: {ENV}. Must be one of {valid_envs}. Exiting notebook.")
    dbutils.notebook.exit("Invalid ENV")

# Validate PROTOCOL
if PROTOCOL not in valid_protocols:
    print(f"Invalid PROTOCOL: {PROTOCOL}. Must be one of {valid_protocols}. Exiting notebook.")
    dbutils.notebook.exit("Invalid PROTOCOL")
    
bronze_schema = f"fpl_bronze_{ENV}"
silver_schema = f"fpl_silver_{ENV}"
historic_seasons = [f"{str(y)[2:]}_{str(y+1)[-2:]}" for y in range(2016, 2025)]
api_seasons = ["25_26"]

#Get Players and Write to Silver

For historic seasons, player data is in players_raw_{season} and contains data such as name, element_type (position). 

For API seasons, this is in elements_{season}.

players_gameweek_stats_{season} contains weekly stats per player. Using this and fixtures, can derive the team of the player each week, as well as FPL value (price).

In [0]:
if PROTOCOL == "HIST":
    df_historic = get_players(bronze_schema, silver_schema, historic_seasons, source_type="HIST")
    df_api = get_players(bronze_schema, silver_schema, api_seasons, source_type="API")
    players_df = df_historic.unionByName(df_api)

    write_to_table(
        df = players_df,
        table_name = f"{silver_schema}.players",
        mode =  "overwrite",
        merge_schema = False
    ) 

elif PROTOCOL == "INCR":
    players_df = get_players(bronze_schema, silver_schema, api_seasons, source_type="API")

    merge_to_table(
        df = players_df,
        table_name = f"{silver_schema}.players",
        merge_condition = "target.player_season_key = source.player_season_key AND target.team_key = source.team_key",
        spark = spark
    )